実際にあるKeypointのみ出力  
端点からノードを少し延長して描画

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import numpy as np
import shutil

json_file_path = '../images/0000_base/0000_base.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

output_dir = './kp_images'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

circle_size = 10  # Keypointの丸の大きさ
line_width = 4   # 直線の太さ
extend_length = 20  # 直線を延長する長さ

for image_id, annotations in annotations_by_image.items():
    fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels

    for annotation in annotations:
        keypoints = annotation['keypoints']
        keypoints_xy = [(keypoints[i], keypoints[i+1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i+1] != 0]

        for i, (x, y) in enumerate(keypoints_xy):
            ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
            if i > 0:
                prev_x, prev_y = keypoints_xy[i-1]
                dx = x - prev_x
                dy = y - prev_y
                length = np.sqrt(dx**2 + dy**2)
                extend_dx = (dx / length) * extend_length
                extend_dy = (dy / length) * extend_length

                if i == 1:
                    extended_prev_x = prev_x - extend_dx
                    extended_prev_y = prev_y - extend_dy
                    ax.plot([extended_prev_x, x], [extended_prev_y, y], 'k-', linewidth=line_width)
                elif i == len(keypoints_xy) - 1:
                    extended_x = x + extend_dx
                    extended_y = y + extend_dy
                    ax.plot([prev_x, extended_x], [prev_y, extended_y], 'k-', linewidth=line_width)
                else:
                    ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

    width = 512
    height = 512

    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')

    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    output_path = os.path.join(output_dir, f'keypoints_image_{image_id}.png')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

print(f'Images are saved in the directory: {output_dir}')


実際にあるKeypointと異なる画像のKeypointを3つ組み合わせた画像を出力

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import numpy as np
import shutil
import random

combin_kp_num = 1000  # 1000枚出力に変更

json_file_path = '../images/0000_base/0000_base.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

output_dir = './kp_images'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

combined_output_dir = './combined_kp_images'
if os.path.exists(combined_output_dir):
    shutil.rmtree(combined_output_dir)
os.makedirs(combined_output_dir, exist_ok=True)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

circle_size = 10  # Keypointの丸の大きさ
line_width = 4   # 直線の太さ
extend_length = 20  # 直線を延長する長さ

# 現在の出力
for image_id, annotations in annotations_by_image.items():
    fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels

    for annotation in annotations:
        keypoints = annotation['keypoints']
        keypoints_xy = [(keypoints[i], keypoints[i+1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i+1] != 0]

        for i, (x, y) in enumerate(keypoints_xy):
            ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
            if i > 0:
                prev_x, prev_y = keypoints_xy[i-1]
                dx = x - prev_x
                dy = y - prev_y
                length = np.sqrt(dx**2 + dy**2)
                extend_dx = (dx / length) * extend_length
                extend_dy = (dy / length) * extend_length

                if i == 1:
                    extended_prev_x = prev_x - extend_dx
                    extended_prev_y = prev_y - extend_dy
                    ax.plot([extended_prev_x, x], [extended_prev_y, y], 'k-', linewidth=line_width)
                elif i == len(keypoints_xy) - 1:
                    extended_x = x + extend_dx
                    extended_y = y + extend_dy
                    ax.plot([prev_x, extended_x], [prev_y, extended_y], 'k-', linewidth=line_width)
                else:
                    ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

    width = 512
    height = 512

    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')

    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    output_path = os.path.join(output_dir, f'keypoints_image_{image_id}.png')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

# 異なる画像のKeypoint情報を組み合わせた出力
image_ids = list(annotations_by_image.keys())

for i in range(combin_kp_num):
    selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
    fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels

    for image_id in selected_image_ids:
        annotations = annotations_by_image[image_id]
        annotation = random.choice(annotations)
        keypoints = annotation['keypoints']
        keypoints_xy = [(keypoints[i], keypoints[i+1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i+1] != 0]

        for j, (x, y) in enumerate(keypoints_xy):
            ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
            if j > 0:
                prev_x, prev_y = keypoints_xy[j-1]
                dx = x - prev_x
                dy = y - prev_y
                length = np.sqrt(dx**2 + dy**2)
                extend_dx = (dx / length) * extend_length
                extend_dy = (dy / length) * extend_length

                if j == 1:
                    extended_prev_x = prev_x - extend_dx
                    extended_prev_y = prev_y - extend_dy
                    ax.plot([extended_prev_x, x], [extended_prev_y, y], 'k-', linewidth=line_width)
                elif j == len(keypoints_xy) - 1:
                    extended_x = x + extend_dx
                    extended_y = y + extend_dy
                    ax.plot([prev_x, extended_x], [prev_y, extended_y], 'k-', linewidth=line_width)
                else:
                    ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

    width = 512
    height = 512

    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')

    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    output_path = os.path.join(combined_output_dir, f'combined_keypoints_image_{i+1}.png')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

print("done")


In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import numpy as np
import shutil
import random

combin_kp_num = 1000  # 1000枚出力に変更

json_file_path = '/mnt/data/test.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

combined_output_dir = './combined_kp_images'
combined_bbox_output_dir = './combined_bbox_images'

def setup_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

setup_directory(combined_output_dir)
setup_directory(combined_bbox_output_dir)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

circle_size = 10  # Keypointの丸の大きさ
line_width = 4   # 直線の太さ

def draw_keypoints(ax, keypoints):
    keypoints_xy = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i + 1] != 0]
    for i, (x, y) in enumerate(keypoints_xy):
        ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
        if i > 0:
            prev_x, prev_y = keypoints_xy[i - 1]
            ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

def draw_bbox(ax, bbox):
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=1, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

def save_plot(fig, ax, output_path):
    width = 512
    height = 512
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

def generate_combined_keypoints_images():
    image_ids = list(annotations_by_image.keys())
    for i in range(combin_kp_num):
        selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for image_id in selected_image_ids:
            annotations = annotations_by_image[image_id]
            annotation = random.choice(annotations)
            keypoints = annotation['keypoints']
            draw_keypoints(ax, keypoints)
        output_path = os.path.join(combined_output_dir, f'combined_keypoints_image_{i + 1}.png')
        save_plot(fig, ax, output_path)

def generate_combined_bbox_images_and_annotations():
    image_ids = list(annotations_by_image.keys())
    new_annotations = []
    new_images = []
    max_image_id = max([img['id'] for img in data['images']])
    max_annotation_id = max([ann['id'] for ann in data['annotations']])
    annotation_id = max_annotation_id + 1
    for i in range(combin_kp_num):
        combined_image_id = max_image_id + i + 1
        selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for image_id in selected_image_ids:
            annotations = annotations_by_image[image_id]
            annotation = random.choice(annotations)
            keypoints = annotation['keypoints']
            bbox = annotation['bbox']
            draw_keypoints(ax, keypoints)
            draw_bbox(ax, bbox)
            new_annotations.append({
                "id": annotation_id,
                "image_id": combined_image_id,
                "category_id": annotation['category_id'],
                "bbox": bbox,
                "keypoints": keypoints,
                "num_keypoints": annotation['num_keypoints']
            })
            annotation_id += 1
        output_path = os.path.join(combined_bbox_output_dir, f'combined_bbox_image_{i + 1}.png')
        save_plot(fig, ax, output_path)
        new_images.append({
            "id": combined_image_id,
            "file_name": f'combined_bbox_image_{i + 1}.png',
            "width": 512,
            "height": 512,
            "dataset_id": 0,
            "category_ids": [],
            "path": output_path,
            "annotated": True,
            "annotating": [],
            "num_annotations": len(selected_image_ids),
            "metadata": {},
            "milliseconds": 0,
            "events": [],
            "regenerate_thumbnail": False,
            "is_modified": False
        })
    
    return new_images, new_annotations

generate_combined_keypoints_images()
new_combined_images, new_combined_annotations = generate_combined_bbox_images_and_annotations()

# 新しいJSONファイルに保存
updated_data = {
    "images": new_combined_images,
    "annotations": new_combined_annotations,
    "categories": data['categories']
}

with open('/mnt/data/updated_annotations.json', 'w') as f:
    json.dump(updated_data, f, indent=4)

print("done")


keypoint延長しないバージョン

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import numpy as np
import shutil
import random

combin_kp_num = 1000  # 1000枚出力に変更

json_file_path = '../images/0000_base/0000_base.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

output_dir = './kp_images'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

combined_output_dir = './combined_kp_images'
if os.path.exists(combined_output_dir):
    shutil.rmtree(combined_output_dir)
os.makedirs(combined_output_dir, exist_ok=True)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

circle_size = 10  # Keypointの丸の大きさ
line_width = 4   # 直線の太さ

# 現在の出力
for image_id, annotations in annotations_by_image.items():
    fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels

    for annotation in annotations:
        keypoints = annotation['keypoints']
        keypoints_xy = [(keypoints[i], keypoints[i+1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i+1] != 0]

        for i, (x, y) in enumerate(keypoints_xy):
            ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
            if i > 0:
                prev_x, prev_y = keypoints_xy[i-1]
                ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

    width = 512
    height = 512

    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')

    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    output_path = os.path.join(output_dir, f'keypoints_image_{image_id}.png')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

# 異なる画像のKeypoint情報を組み合わせた出力
image_ids = list(annotations_by_image.keys())

for i in range(combin_kp_num):
    selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
    fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels

    for image_id in selected_image_ids:
        annotations = annotations_by_image[image_id]
        annotation = random.choice(annotations)
        keypoints = annotation['keypoints']
        keypoints_xy = [(keypoints[i], keypoints[i+1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i+1] != 0]

        for j, (x, y) in enumerate(keypoints_xy):
            ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
            if j > 0:
                prev_x, prev_y = keypoints_xy[j-1]
                ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

    width = 512
    height = 512

    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')

    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    output_path = os.path.join(combined_output_dir, f'combined_keypoints_image_{i+1}.png')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

print("done")


In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import numpy as np
import shutil
import random

combin_kp_num = 1000  # 1000枚出力に変更

json_file_path = '../images/0000_base/0000_base.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

output_dir = './kp_images'
bbox_output_dir = './bbox_images'
combined_output_dir = './combined_kp_images'
combined_bbox_output_dir = './combined_bbox_images'

def setup_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

setup_directory(output_dir)
setup_directory(bbox_output_dir)
setup_directory(combined_output_dir)
setup_directory(combined_bbox_output_dir)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

circle_size = 10  # Keypointの丸の大きさ
line_width = 4   # 直線の太さ

def draw_keypoints(ax, keypoints):
    keypoints_xy = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i + 1] != 0]
    for i, (x, y) in enumerate(keypoints_xy):
        ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
        if i > 0:
            prev_x, prev_y = keypoints_xy[i - 1]
            ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

def draw_bbox(ax, bbox):
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=1, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

def save_plot(ax, output_path):
    width = 512
    height = 512
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

def generate_keypoints_images():
    for image_id, annotations in annotations_by_image.items():
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for annotation in annotations:
            keypoints = annotation['keypoints']
            draw_keypoints(ax, keypoints)
        output_path = os.path.join(output_dir, f'keypoints_image_{image_id}.png')
        save_plot(ax, output_path)

def generate_bbox_images():
    for image_id, annotations in annotations_by_image.items():
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for annotation in annotations:
            keypoints = annotation['keypoints']
            bbox = annotation['bbox']
            draw_keypoints(ax, keypoints)
            draw_bbox(ax, bbox)
        output_path = os.path.join(bbox_output_dir, f'bbox_image_{image_id}.png')
        save_plot(ax, output_path)

def generate_combined_keypoints_images():
    image_ids = list(annotations_by_image.keys())
    for i in range(combin_kp_num):
        selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for image_id in selected_image_ids:
            annotations = annotations_by_image[image_id]
            annotation = random.choice(annotations)
            keypoints = annotation['keypoints']
            draw_keypoints(ax, keypoints)
        output_path = os.path.join(combined_output_dir, f'combined_keypoints_image_{i + 1}.png')
        save_plot(ax, output_path)

def generate_combined_bbox_images():
    image_ids = list(annotations_by_image.keys())
    for i in range(combin_kp_num):
        selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for image_id in selected_image_ids:
            annotations = annotations_by_image[image_id]
            annotation = random.choice(annotations)
            keypoints = annotation['keypoints']
            bbox = annotation['bbox']
            draw_keypoints(ax, keypoints)
            draw_bbox(ax, bbox)
        output_path = os.path.join(combined_bbox_output_dir, f'combined_bbox_image_{i + 1}.png')
        save_plot(ax, output_path)

generate_keypoints_images()
generate_bbox_images()
generate_combined_keypoints_images()
generate_combined_bbox_images()

print("done")


枚数指定してbboxも合わせて描画するバージョン

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import numpy as np
import shutil
import random

combin_kp_num = 1000  # 1000枚出力に変更

json_file_path = '../images/0000_base/0000_base.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

combined_output_dir = './combined_kp_images'
combined_bbox_output_dir = './combined_bbox_images'

def setup_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

setup_directory(combined_output_dir)
setup_directory(combined_bbox_output_dir)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

circle_size = 10  # Keypointの丸の大きさ
line_width = 4   # 直線の太さ

def draw_keypoints(ax, keypoints):
    keypoints_xy = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i + 1] != 0]
    for i, (x, y) in enumerate(keypoints_xy):
        ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
        if i > 0:
            prev_x, prev_y = keypoints_xy[i - 1]
            ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

def draw_bbox(ax, bbox):
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=1, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

def save_plot(fig, ax, output_path):
    width = 512
    height = 512
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

def generate_combined_keypoints_images():
    image_ids = list(annotations_by_image.keys())
    for i in range(combin_kp_num):
        selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for image_id in selected_image_ids:
            annotations = annotations_by_image[image_id]
            annotation = random.choice(annotations)
            keypoints = annotation['keypoints']
            draw_keypoints(ax, keypoints)
        output_path = os.path.join(combined_output_dir, f'combined_keypoints_image_{i + 1}.png')
        save_plot(fig, ax, output_path)

def generate_combined_bbox_images_and_annotations():
    image_ids = list(annotations_by_image.keys())
    new_annotations = []
    new_images = []
    max_image_id = max([img['id'] for img in data['images']])
    max_annotation_id = max([ann['id'] for ann in data['annotations']])
    annotation_id = max_annotation_id + 1
    for i in range(combin_kp_num):
        combined_image_id = max_image_id + i + 1
        selected_image_ids = random.choices(image_ids, k=3)  # 再利用のためにchoicesを使用
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        for image_id in selected_image_ids:
            annotations = annotations_by_image[image_id]
            annotation = random.choice(annotations)
            keypoints = annotation['keypoints']
            bbox = annotation['bbox']
            draw_keypoints(ax, keypoints)
            draw_bbox(ax, bbox)
            new_annotations.append({
                "id": annotation_id,
                "image_id": combined_image_id,
                "category_id": annotation['category_id'],
                "bbox": bbox,
                "keypoints": keypoints,
                "num_keypoints": annotation['num_keypoints']
            })
            annotation_id += 1
        output_path = os.path.join(combined_bbox_output_dir, f'combined_bbox_image_{i + 1}.png')
        save_plot(fig, ax, output_path)
        new_images.append({
            "id": combined_image_id,
            "file_name": f'combined_bbox_image_{i + 1}.png',
            "width": 512,
            "height": 512,
            "dataset_id": 0,
            "category_ids": [],
            "path": output_path,
            "annotated": True,
            "annotating": [],
            "num_annotations": len(selected_image_ids),
            "metadata": {},
            "milliseconds": 0,
            "events": [],
            "regenerate_thumbnail": False,
            "is_modified": False
        })
    
    return new_images, new_annotations

generate_combined_keypoints_images()
new_combined_images, new_combined_annotations = generate_combined_bbox_images_and_annotations()

# 新しいJSONファイルに保存
updated_data = {
    "images": new_combined_images,
    "annotations": new_combined_annotations,
    "categories": data['categories']
}

with open('./updated_annotations.json', 'w') as f:
    json.dump(updated_data, f, indent=4)

print("done")


jsonファイルのアノテーションデータと一対一対応

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import shutil

num = "02"

def setup_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

#combined_output_dir = '../images/no_extend/'+ num + '/line2line'
combined_output_dir = './mask'
#combined_bbox_output_dir = '../images/no_extend/'+num+'/overall'
combined_bbox_output_dir = './source'
setup_directory(combined_output_dir)
setup_directory(combined_bbox_output_dir)

#json_file_path = '../images/gen_data/'+num+'_gen_coco.json'
json_file_path = './valid.json'

with open(json_file_path, 'r') as f:
    data = json.load(f)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

def draw_keypoints(ax, keypoints, circle_size, line_width):
    keypoints_xy = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i + 1] != 0]
    for i, (x, y) in enumerate(keypoints_xy):
        ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
        if i > 0:
            prev_x, prev_y = keypoints_xy[i - 1]
            ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

def draw_bbox(ax, bbox):
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=1, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

def save_plot(fig, ax, output_path):
    width = 512
    height = 512
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

def generate_keypoints_line_images(circle_size, line_width):
    for image_info in data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        if image_id in annotations_by_image:
            for annotation in annotations_by_image[image_id]:
                keypoints = annotation['keypoints']
                draw_keypoints(ax, keypoints, circle_size, line_width)
        output_path = os.path.join(combined_output_dir, file_name)
        save_plot(fig, ax, output_path)

def generate_keypoints_bbox_images(circle_size, line_width):
    for image_info in data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        if image_id in annotations_by_image:
            for annotation in annotations_by_image[image_id]:
                keypoints = annotation['keypoints']
                bbox = annotation['bbox']
                draw_keypoints(ax, keypoints, circle_size, line_width)
                draw_bbox(ax, bbox)
        output_path = os.path.join(combined_bbox_output_dir, file_name)
        save_plot(fig, ax, output_path)

# line2line
keypoints_line_circle_size = 8
keypoints_line_width = 5

# overall
keypoints_bbox_circle_size = 6
keypoints_bbox_line_width = 2

generate_keypoints_line_images(keypoints_line_circle_size, keypoints_line_width)
generate_keypoints_bbox_images(keypoints_bbox_circle_size, keypoints_bbox_line_width)

import os
from PIL import Image, ImageOps

def invert_images_in_directory(directory):
    image_files = [f for f in os.listdir(directory) if f.endswith('.jpg')]
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path)
        inverted_image = ImageOps.invert(image.convert('RGB'))
        inverted_image.save(image_path)

invert_images_in_directory(combined_output_dir)
invert_images_in_directory(combined_bbox_output_dir)



端点を延長するパターン

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import shutil
import numpy as np
from PIL import Image, ImageOps

num = "05"

def setup_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

combined_output_dir = '../images/gen_data/' + num + '/line2line'
combined_bbox_output_dir = '../images/gen_data/' + num + '/overall'
setup_directory(combined_output_dir)
setup_directory(combined_bbox_output_dir)

json_file_path = '../images/gen_data/' + num + '_gen_coco.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

def draw_keypoints(ax, keypoints, circle_size, line_width, extend_length=10, mode='line2line'):
    keypoints_xy = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i + 1] != 0]
    for j, (x, y) in enumerate(keypoints_xy):
        ax.plot(x, y, 'ko', markersize=circle_size)  # 'ko' means black color, circle marker
        if j > 0:
            prev_x, prev_y = keypoints_xy[j-1]
            dx = x - prev_x
            dy = y - prev_y
            length = np.sqrt(dx**2 + dy**2)
            extend_dx = (dx / length) * extend_length
            extend_dy = (dy / length) * extend_length

            if mode == 'line2line':
                if j == 1:
                    extended_prev_x = prev_x - extend_dx
                    extended_prev_y = prev_y - extend_dy
                    ax.plot([extended_prev_x, x], [extended_prev_y, y], 'k-', linewidth=line_width)
                elif j == len(keypoints_xy) - 1:
                    extended_x = x + extend_dx
                    extended_y = y + extend_dy
                    ax.plot([prev_x, extended_x], [prev_y, extended_y], 'k-', linewidth=line_width)
                else:
                    ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)
            else:
                ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)

def draw_bbox(ax, bbox):
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=1, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

def save_plot(fig, ax, output_path):
    width = 512
    height = 512
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)  # 100 DPI * 5.12 inches = 512 pixels
    plt.close(fig)

def generate_keypoints_line_images(circle_size, line_width, extend_length):
    for image_info in data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        if image_id in annotations_by_image:
            for annotation in annotations_by_image[image_id]:
                keypoints = annotation['keypoints']
                draw_keypoints(ax, keypoints, circle_size, line_width, extend_length=extend_length, mode='line2line')
        output_path = os.path.join(combined_output_dir, file_name)
        save_plot(fig, ax, output_path)

def generate_keypoints_bbox_images(circle_size, line_width):
    for image_info in data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512 pixels
        if image_id in annotations_by_image:
            for annotation in annotations_by_image[image_id]:
                keypoints = annotation['keypoints']
                bbox = annotation['bbox']
                draw_keypoints(ax, keypoints, circle_size, line_width, mode='bbox')
                draw_bbox(ax, bbox)
        output_path = os.path.join(combined_bbox_output_dir, file_name)
        save_plot(fig, ax, output_path)

# line2line
keypoints_line_circle_size = 8
keypoints_line_width = 5
extend_length = 15

# overall
keypoints_bbox_circle_size = 8
keypoints_bbox_line_width = 2

generate_keypoints_line_images(keypoints_line_circle_size, keypoints_line_width, extend_length)
generate_keypoints_bbox_images(keypoints_bbox_circle_size, keypoints_bbox_line_width)

def invert_images_in_directory(directory):
    image_files = [f for f in os.listdir(directory) if f.endswith('.jpg')]
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path)
        inverted_image = ImageOps.invert(image.convert('RGB'))
        inverted_image.save(image_path)

invert_images_in_directory(combined_output_dir)
invert_images_in_directory(combined_bbox_output_dir)


特定のノードをランダムでひとつ削除  
overallとline2lineを対応付けて葉の遮蔽を制御  
葉の遮蔽が発生しているか入出力画像の一致度により判定

In [ ]:
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import shutil

num = "05"

def setup_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)

combined_output_dir = '../images/test2/'+ num + '/line2line'
combined_bbox_output_dir = '../images/test2/'+num+'/overall'
setup_directory(combined_output_dir)
setup_directory(combined_bbox_output_dir)

json_file_path = '../images/gen_data/'+num+'_gen_coco.json'
with open(json_file_path, 'r') as f:
    data = json.load(f)

annotations_by_image = defaultdict(list)
for annotation in data['annotations']:
    image_id = annotation['image_id']
    annotations_by_image[image_id].append(annotation)

def draw_keypoints_with_line_removed(ax, keypoints, circle_size, line_width):
    keypoints_xy = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3) if keypoints[i] != 0 and keypoints[i + 1] != 0]
    if len(keypoints_xy) < 3:
        return
    remove_line_index = len(keypoints_xy) - 2
    
    for i, (x, y) in enumerate(keypoints_xy):
        ax.plot(x, y, 'ko', markersize=circle_size)
        
        if i > 0:
            prev_x, prev_y = keypoints_xy[i - 1]
            if i == remove_line_index:
                mid_x = (prev_x + x) / 2
                mid_y = (prev_y + y) / 2
                # 全部消す 端から半分消すなら
                # ax.plot([mid_x, x], [mid_y, y], 'k-', linewidth=line_width)
                # ax.plot([prev_x, mid_x], [prev_y, mid_y], 'k-', linewidth=line_width)
                # 中央から半分消す
                # ax.plot([(mid_x+x)/2, x], [(mid_y+y)/2, y], 'k-', linewidth=line_width)
                # ax.plot([prev_x, (mid_x+prev_x)/2], [prev_y, (mid_y+prev_y)/2], 'k-', linewidth=line_width)
                
            else:
                ax.plot([prev_x, x], [prev_y, y], 'k-', linewidth=line_width)
                

def draw_bbox(ax, bbox):
    bbox_x, bbox_y, bbox_w, bbox_h = bbox
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, linewidth=1, edgecolor='k', facecolor='none')
    ax.add_patch(rect)

def save_plot(fig, ax, output_path):
    width = 512
    height = 512
    ax.set_xlim(0, width)
    ax.set_ylim(height, 0)
    ax.set_facecolor('white')
    ax.axis('off')
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

def generate_keypoints_line_images(circle_size, line_width):
    for image_info in data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512ピクセル
        if image_id in annotations_by_image:
            for annotation in annotations_by_image[image_id]:
                keypoints = annotation['keypoints']
                draw_keypoints_with_line_removed(ax, keypoints, circle_size, line_width)
        output_path = os.path.join(combined_output_dir, file_name)
        save_plot(fig, ax, output_path)

def generate_keypoints_bbox_images(circle_size, line_width):
    for image_info in data['images']:
        image_id = image_info['id']
        file_name = image_info['file_name']
        fig, ax = plt.subplots(figsize=(5.12, 5.12))  # 512x512ピクセル
        if image_id in annotations_by_image:
            for annotation in annotations_by_image[image_id]:
                keypoints = annotation['keypoints']
                bbox = annotation['bbox']
                draw_keypoints_with_line_removed(ax, keypoints, circle_size, line_width)
                draw_bbox(ax, bbox)
        output_path = os.path.join(combined_bbox_output_dir, file_name)
        save_plot(fig, ax, output_path)

# line2line
keypoints_line_circle_size = 8
keypoints_line_width = 5

# overall
keypoints_bbox_circle_size = 8
keypoints_bbox_line_width = 2

generate_keypoints_line_images(keypoints_line_circle_size, keypoints_line_width)
generate_keypoints_bbox_images(keypoints_bbox_circle_size, keypoints_bbox_line_width)

import os
from PIL import Image, ImageOps

def invert_images_in_directory(directory):
    image_files = [f for f in os.listdir(directory) if f.endswith('.jpg')]
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path)
        inverted_image = ImageOps.invert(image.convert('RGB'))
        inverted_image.save(image_path)

invert_images_in_directory(combined_output_dir)
invert_images_in_directory(combined_bbox_output_dir)